# Search engines

This notebook shows how to use [PyTerrier](https://github.com/terrier-org/pyterrier) on the [CORD19 corpus](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7251955/) and the [TREC Covid test collection](https://ir.nist.gov/covidSubmit/).
Hereafter we are going to see how to:
- index a collection
- access an index
- search an index
- compare the performances of indexing approaches
- learn ranking
- evaluate ranking

The notebook in mainly based on the tutorials at this [link](https://github.com/terrier-org/cikm2021tutorial/tree/main/notebooks), which are part of the tutorial series "[IR From Bag-of-words to BERT and Beyond through Practical Experiments](https://github.com/terrier-org/cikm2021tutorial/)" created for the [CIKM 2021](https://www.cikm2021.org/).

## Tools installation and configuration

PyTerrier is a Python framework, but uses the underlying [Terrier information retrieval toolkit](http://terrier.org) for many indexing and retrieval operations. While PyTerrier was new in 2020, Terrier is written in Java and has a long history dating back to 2001. PyTerrier makes it easy to perform IR experiments in Python, but using the mature Terrier platform for the expensive indexing and retrieval operations. 

PyTerrier is installed as follows. This might take a few minutes, in the meanwhile you can take a look at [PyTerrier documentation](https://pyterrier.readthedocs.io/en/latest/).

In [7]:
#!pip install -q python-terrier

The next step is to initialise PyTerrier. This is performed using PyTerrier's `init()` method. The `init()` method is needed as PyTerrier must download Terrier's jar file and start the Java virtual machine. We prevent `init()` from being called more than once by checking `started()`.

In [8]:
import pyterrier as pt
if not pt.started():
  pt.init()

RuntimeError: 
        Unable to find libjvm.so, (tried ['/usr/bin/java/jre/lib/jli/libjli.dylib', '/usr/bin/java/lib/jli/libjli.dylib', '/usr/bin/java/lib/libjli.dylib'])
        you can use the JVM_PATH env variable with the absolute path
        to libjvm.so to override this lookup, if you know
        where pyjnius should look for it.

        e.g:
            export JAVA_HOME=/usr/lib/jvm/java-8-oracle/
            export JVM_PATH=/usr/lib/jvm/java-8-oracle/jre/lib/amd64/server/libjvm.so
            # run your program
        

## Introduction to document indexing and searching

Much of PyTerrier's view of the world is wrapped up in Pandas dataframes. Let's consider some textual documents in a dataframe.

We can start importing Pandas, a well known Python library to manage tabular data. 
You can find Pandas' documentation at the following [link](https://pandas.pydata.org).


In [ ]:
import pandas as pd

We can set the length of the displayed output to avoid truncating too much text

In [ ]:
pd.set_option('display.max_colwidth', 150)

### Documents

Finally we can create some sample documents to store in a `DataFrame

In [ ]:
docs_df = pd.DataFrame(
    [
        ["d1", "This is the first article in my collection of articles."],
        ["d2", "This is another article in my collection."],
        ["d3", "The topic of this third article is unknown."]
    ], 
    columns=["docno", "text"]
)
docs_df

### Indexing

Before any search engine can estimate which documents are most likely to be relevant for a given query, it must index the documents. 

In the following cell, we index the dataframe's documents. The index, with all its data structures, is written into a directory that we will call `index_3docs`. You can see it in the Google colab files folder on the left.

In [ ]:
indexer = pt.DFIndexer("./index_3docs", overwrite=True)
index_ref = indexer.index(docs_df["text"], docs_df["docno"])
index_ref.toString()

`IndexRef` provides the location where the index is stored. Indeed, we can look in the `index_3docs` directory and see that it has created various files: 

In [ ]:
!ls -lh index_3docs/

We can now load the index:

In [ ]:
index = pt.IndexFactory.of(index_ref)

This is a Terrier index structure, which provides methods such as:
 - `getCollectionStatistics()`
 - `getInvertedIndex()`
 - `getLexicon()`

Note Terrier is written in Java. (Yes, yes... I know... Java... 🤢) 
The Javadoc documentation is here: 
http://terrier.org/docs/current/javadoc/org/terrier/structures/Index.html

Let's see what is returned by the `CollectionStatistics()` method:

In [ ]:
print(index.getCollectionStatistics().toString())

Ok, that seems fair – we have 3 documents. But why only 6 terms? 
Let's check the [`Lexicon`](http://terrier.org/docs/current/javadoc/org/terrier/structures/Lexicon.html), which is our vocabulary. The `Lexicon` contains a set of terms, so we need to iterate over it:

In [ ]:
for kv in index.getLexicon():
    print(kv.getKey(),": ", kv.getValue().toString())

We see the terms and the statistics collected about each.
Note that stopwords were removed and Porter's stemmer has been applied.
Here:
 - `Nt` is the number of unique documents that each term occurs in – this is useful for calculating IDF.
 - `TF` is the total number of occurrences of term in corpus.

Finally, we can use the square bracket notation to lookup terms in the lexicon:


In [ ]:
index.getLexicon()["articl"].toString()

And we can see how many times this term is used in each document that it occurs in (by iterating over the posting lists):

In [ ]:
pointer = index.getLexicon()["articl"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(f'{posting.toString()} doclen = {posting.getDocumentLength()}')

We can see that `"article"` occurs in each of the three documents. 

### Searching an Index

Now that we have indexed our documents, we can run a search over the document collection:

In [ ]:
query = "articles"

br = pt.BatchRetrieve(index, wmodel="TF_IDF")
br.search(query)

Here we used the TF-IDF weighting formula to rank the results. 

The `search()` method returns a dataframe with columns:
 - `qid`: the query identifier
 - `docid`: integer identifier for document 
 - `docno`: string identifier for document
 - `rank`: rank position
 - `score`: tf-idf score
 - `query`: the input query

### Multiple searches

We can also run multiple queries at once:

In [ ]:
queries = pd.DataFrame([["query1", "articles"], ["query2", "first article"], ["query3", "unknown"]], columns=["qid", "query"])
br(queries)

## Loading a real dataset

We'll now load a real dataset of COVID-19 related scientific articles (called CORD19) that is available as an example in PyTerrier

In [ ]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')

We just downloaded the data. Now we index and save the dataset.

The CORD19 corpus contains articles about the COVID-19, we are going to retrieve only the abstract of these articles to compose our documents. The indexing will take a while to run since there are almost 200,000 articles. 

In [ ]:
pt_index_path = './terrier_cord19'
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True, meta_reverse=[])
index_ref = indexer.index(cord19.get_corpus_iter(), fields=('abstract',), meta=('docno',))

### Querying the index

Now load the index and print the statistics:

In [ ]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Let's run a query:

In [ ]:
query = "chemical reactions"

tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tfidf.search(query)

Instead of ranking documents by their TF-IDF score, we could also use the BM25 score which is known to produce high quality search results.

In [ ]:
query = "chemical reactions"

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25.search(query)

Note that the first 5 documents in the ranking are the same as they were for TF-IDF. 

### Evaluating retrieval

So far, we have been creating search engine models, but we haven't decided if any of them are actually any good. In order to determine how good a ranking is, we need annotations telling us which documents are actually relevant for a particular query. 

The CORD19 dataset contains set of queries and relevance assessments (aka qrels)for this purpose. 

For historical reasons, the queries are called "topics". Let's have a look at the first 10 queries:

In [ ]:
queries = cord19.get_topics(variant='title')
queries.head(10)

And the first 10 relevance judgements (called qrels):

In [ ]:
qrels = cord19.get_qrels()
qrels.head(10)

We can give a look at the distribution of the relevance scores in the data set. To do so we train the count of occurrences of each label, sort the values by label and plot these values in a bat plot (note the log-scale on the y-axis):

In [ ]:
qrels['label'].value_counts().plot(kind='bar', log=True)

Note that there are some relevance judgements taking the value of -1. In the original documentation of the data set (https://ir-datasets.com/cord19.html) only the values 0, 1, and 2 are accepted, so the -1 must be errors in the annotation, since they are only 2 we can drop the corresponding qrels.

To drop the undesired we simply have to retain all the qrels with a label different from -1

In [ ]:
qrels = qrels[qrels['label'] != -1]

The qrels contain information on the relevance labels for the query-document pairs. To do our evaluation we need to load also the queries we are going to use to test our retrieval system. For historical reasons, queries are called "topics" in this context.

Let's collect the topics in the CORD-19 corpus:

In [ ]:
topics = cord19.get_topics(variant='title')
topics

Now let us use these queries and relevance judgements to compare different retrieval functions to see how well they perform at ranking documents in the collection. We compare BM25 and TF-IDF in terms of two common ranking evaluation measures (MAP and NDCG), where higher values indicate a better ranking:  

In [ ]:
pt.Experiment(
    [tfidf, bm25],
    topics,
    qrels,
    eval_metrics=["map", "ndcg"]
)

How were those scores calculated? We can have a look at the score and corresponding relevance labels for individual query document pairs as follows:

In [ ]:
# Rank the documents for each query using the TF-IDF scoring function:
results = tfidf(cord19.get_topics(variant='title'))
# Add the relevance labels (qrels) to the table:
results = results.merge(qrels, on=["qid", "docno"], how="left").fillna(0)
# Display the output 
results

## Pipelines

For this part of the notebook, we'll be using a different (pre-built) Terrier index with term position information, which was built uisng the following code. (We wont run it now since it would take a while to run.)
```python
pt_index_path = './terrier_cord19_blocks'
indexer = pt.index.IterDictIndexer(pt_index_path blocks=True)
index_ref = indexer.index(cord19.get_corpus_iter(), fields=('abstract',), meta=('docno',))
```

However, its just as quick to use the pre-built index from the Terrier Data Repository. We use the ['terrier_stemmed_positions'](http://data.terrier.org/trec-covid.dataset.html#terrier_stemmed_positions) index variant.

In [ ]:
index = pt.IndexFactory.of(
    pt.get_dataset('trec-covid').get_index('terrier_stemmed_positions')
)

#### Operators

`BatchRetrieve` objects can be combined using some special operators. These combinations are called pipelines. 

Hereafter we are going to see three operators:
- Composition
- Rank cut-off
- Union

Before moving to the actual operators se define three retrievers using the index we have just created. We are going to define retrievers using 
- TF
- TF-IDF
- BM25

In [ ]:
tf = pt.BatchRetrieve(index, wmodel="Tf")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

##### Composition

The first operator we are going to see is the composition. This operator allow to re-rank the output of one retriever using a second retriever. 

To use it, we simply need to compose two retrievers using the  `>>` operator. 

Given the first query in the topics we loaded before

In [ ]:
query = 'chemical'

Let's see first what TF alone would retrieve

In [ ]:
tf.search(query)

Now let's compose the TF with the BM25, in this way we will re-rank the TF-IDF results using BM25.

First we create a composition an then run the search using the composition pipeline.

In [ ]:
composition = tf >> bm25

composition.search(query)

As you can see some documents have been further re-ordered by the BM25

##### Rank cut-off

The second operator we are going to see is the rank cut-off. This operator allow to retain only the top *n* results. 

To use it, we simply need to compose a new retriever applying the `% n` operator to an existing retriever (with *n* being the number of results we want to retain. This is useful to do an early pruning of the the retreived results.

Let's retain the top 10 results using the TF-IDF score.

First we define the rank cut-off pipeline using the TF retriever, then we use it to answer the query.

In [ ]:
tf_idf_cut = tf_idf % 10

tf_idf_cut.search(query)

Of course these pipeline operators can be combined. For example we can re-rank with BM25 only the first 20 results from TF.

Again we define first our pipeline with the two operations and then we search the query.

In [ ]:
combination_cut = (tf % 20) >> bm25

combination_cut.search(query)

##### Results union

Finally we can also combine the results from different retreivers, merging them together. In this case we are going to define the pipeline using two retrievers interleaved by the `|` operator.

For example we can combine the top 10 results from TF with the top 10 results from TF-IDF. As usual we define first our pipeline and then we run our query.

In [ ]:
union = (tf % 10) | (tf_idf % 10)

union.search(query)

There are some overlappings between the results of TF and TF-IDf, as a results the total number of retreived documents is lower than 20.
Also nothe that we do not have scores here since TF and TF-IDF are not comparable.

What we can do is add som re-ranking to our pipeline. We can re-score the output given by the union of TF (with cut-off at 10) and TF-IDF (with cut-off at 10) using BM25 

In [ ]:
pipeline = ((tf % 10) | (tf_idf % 10)) >> bm25

pipeline.search(query)

#### Evaluating retireval pipelines

How do these complex retrieval pipelines perform? 

We can compare the MAP and NDCG scores on the topics avaialble on the CORD-19 data set using the qrels we loaded before.

Let's compare
- BM25 (used as baseline)
- TF-IDF re-scored with BM25
- the union of TF-IDF and TF top 500 results re-scored with BM25

In [ ]:
pt.Experiment(
  [bm25, tf_idf >> bm25, ((tf % 500) | (tf_idf % 500)) >> bm25],
  topics,
  qrels,
  eval_metrics=["map", "ndcg"],
  names=["BM25", "TF-IDF >> BM25", "((TF % 10) | (TF-IDF % 10)) >> BM25"]
)

## Learning to Rank

In this last part of the notebook, you will experience constructing, learning, evaluating and analysing learning to rank pipelines.


Firstly, lets split out topics into train, validation and test sets. TREC Covid only has 50 topics, which isnt a lot for training. We'll split this 30 for training 5 for validation and 15 for test. We will also examine statistical significance, even if this is artificial for 15 topics.

We're only going to-rank the top 10 documents for each query - hopefully learning to rank can help to re-rank the top 10 documents to be more effective.

We define some constants to controthe cutoff and make the experiments reproducible 

In [ ]:
RANK_CUTOFF = 10
SEED = 42

Then we use the splitting utility from Scikit-Learn

In [ ]:
from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

### Feature Set

Lets define our feature set.  We're going to have a total of 5 features:

1.   the BM25 abstract score;
2.   the BM25 score on the title;
3.   whether the abstract contain 'coronavirus covid', scored by BM25;
4.   whether the paper was released/published in 2020 (Recent papers were more useful for this task);
5.   whether the paper is a formal publication? (we can get this information checking whether the paper has a special identifier called DOI)

Several of these feature require additional metadata `["title", "date", "doi"]`. Fortunately, the TREC Covid dataset allows us to obtain more metadata after indexing. We use `pt.text.get_text(cord19, ["title", "date", "doi"])` to retrieve these extra metadata columns.

Note the complete pipeline definition is a bit complex, just keep in mind we are interested in extracting the features we just listed

In [ ]:
ltr_feats = (bm25 % RANK_CUTOFF) >> pt.text.get_text(cord19, ["title", "date", "doi"]) >> (
    pt.transformer.IdentityTransformer()
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="title", takes='docs', wmodel='BM25') ) 
    ** # score of text for query 'coronavirus covid'
    (pt.apply.query(lambda row: 'coronavirus covid') >> bm25)
    ** # date 2020
    (pt.apply.doc_score(lambda row: int("2020" in row["date"])))
    ** # has doi
    (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0)))
)

For reference, we create a list with the names of the features

In [ ]:
fnames=["BM25 abstract", "BM25 title", "coronavirus covid", "2020", "DOI"]

Lets see the output for a given query. 

In [ ]:
ltr_feats.search("coronovirus origin")

We can see that we now have extra document metadata columns `["title", "date", "doi"]`, as well as the `"features"` column. This last column contains the array of features we are going to use in the learning step

We can also look at the raw features values (in this case for the first ranked document)

In [ ]:
print('Features:', ltr_feats.search("coronovirus origin").iloc[0]["features"])

### Learning 

In this part of the notebook, we apply three different learning to rank techniques:

 - coordinate ascent from FastRank, a listwise linear technique
 - random forests from `scikit-learn`, a pointwise regression tree technique
 - LambdaMART from LightGBM, a listwise regression tree technique

In each case, we take our feature pipeline, `ltr_feats1`, and we compose it (`>>`) with the learned model. We use `pt.ltr.apply_learned_model()` which knows how to deal with different learners.

The full pipeline is then fitted (learned) using `.fit()`, specifying the training topics and qrels. Importantly, the preceeding stages of the pipeline (retrieval and feature calculation) are applied to the training topics in order to obtained the results, which are then passed to the learning to rank technique. LightGBM has early stopping enabled, which uses a validation topics set – similarly the validation topics are transformed into validation results.


#### Linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr_pipe = ltr_feats >> pt.ltr.apply_learned_model(lr)

lr_pipe.fit(train_topics, qrels)

#### Non-linear regression model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=-1)

rf_pipe = ltr_feats >> pt.ltr.apply_learned_model(rf)

rf_pipe.fit(train_topics, qrels)

#### LambdaMART

LambdaMART is a listwise appraoch to learning to rank documents. It makes use of Gradient Boosted Regression trees to optimize an approximation to the NDCG evaluation function. 

To learn a LambdaMART model we first need to install [LightGBM](https://lightgbm.readthedocs.io/en/latest/).

In [ ]:
!pip install -q --upgrade lightgbm==3.1.1

Now we can train the model:

In [ ]:
import lightgbm as lgb

# This configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

lmart_x_pipe.fit(train_topics, qrels, valid_topics, qrels)

### Evaluation

Lets now compare our ranking pipelines on our 15 topics with the BM25 baseline. 

We'll report MAP, NDCG measures.

In [ ]:
pt.Experiment(
    [bm25 % RANK_CUTOFF, lr_pipe, rf_pipe, lmart_x_pipe],
    test_topics,
    qrels, 
    names=["BM25",  "BM25 + LinearRegression", "BM25 + RandomForest", "BM25 + LambdaMART"],
    eval_metrics=["map", "ndcg"]
)

The non-linear regression model produced the best results

### Analysis

We can also analyze our learned models to understand the role of the different features we are using.

One way to do this analysis is to plot feature weights or importance from the different learned models. For the Linear Regression model, we plot the feature weights, while for the Random Forest and LambdaMART we report the feature importance.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Create figure
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(12, 8))

# Plot Linear Regression model weights
ax0.bar(np.arange(len(fnames)), lr.coef_)
ax0.set_xticks(np.arange(len(fnames)))
ax0.set_xticklabels(fnames, rotation=45, ha='right')
ax0.set_title("Linear Regression")
ax0.set_ylabel("Feature Weights")
# Plot Random Forest feature importance
ax1.bar(np.arange(len(fnames)), rf.feature_importances_)
ax1.set_xticks(np.arange(len(fnames)))
ax1.set_xticklabels(fnames, rotation=45, ha='right')
ax1.set_title("Random Forest")
ax1.set_ylabel("Feature Importance")
# Plot LmbdaMART feature importance
ax2.bar(np.arange(len(fnames)), lmart_l.feature_importances_)
ax2.set_xticks(np.arange(len(fnames)))
ax2.set_xticklabels(fnames, rotation=45, ha='right')
ax2.set_title("$\lambda$MART")
ax2.set_ylabel("Feature Importance")

# Display figure
plt.tight_layout()
plt.show()
